In [ ]:
%load_ext autoreload

%autoreload 2

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Detection & HUD Mask Inspection

Visualize YOLO player detections and verify the HUD mask position on sample frames from the football video. Use this notebook to tune `top_percent` and `bottom_percent` parameters so the mask covers scoreboards and UI overlays without clipping players.

## Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pyrootutils
from matplotlib.patches import Rectangle

from football_tracking_demo.config import load_config
from football_tracking_demo.detector import PlayerDetector, apply_hud_mask
from football_tracking_demo.video_io import get_video_metadata

## Parameters

In [ ]:
root = pyrootutils.setup_root(
    search_from=".",
    indicator="pyproject.toml",
    project_root_env_var=True,
    dotenv=True,
    pythonpath=True,
    cwd=True,
)

VIDEO_PATH = str(root / "data" / "match.mp4")
CONFIG_PATH = str(root / "config" / "config.yaml")

# Frames to sample (indices into the video)
SAMPLE_FRAME_INDICES = [0, 500, 3000, 6000]

# HUD mask parameters to inspect (can override config values here)
HUD_TOP_PERCENT = 0.05
HUD_BOTTOM_PERCENT = 0.10

## Load Video Metadata

Check basic video properties and confirm the file is accessible.

In [ ]:
meta = get_video_metadata(VIDEO_PATH)
meta

## Extract Sample Frames

Pull a handful of frames at different points in the video for inspection.

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
frames = {}
max_idx = max(SAMPLE_FRAME_INDICES)

for i in range(max_idx + 1):
    ret, frame = cap.read()
    if not ret:
        break
    if i in SAMPLE_FRAME_INDICES:
        frames[i] = frame

cap.release()
print(f"Loaded {len(frames)} sample frames: {sorted(frames.keys())}")

## HUD Mask Overlay Visualization

Show each sample frame with the HUD mask regions highlighted in semi-transparent red. The top and bottom strips indicate pixels that will be blacked out before detection. Verify these regions cover the scoreboard / minimap without clipping actual players on the field.

In [ ]:
def show_hud_overlay(frame_bgr, top_pct, bottom_pct, title=""):
    """Display a frame with HUD mask regions highlighted in red."""
    rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    h, w = rgb.shape[:2]
    top_px = int(h * top_pct)
    bottom_px = int(h * (1.0 - bottom_pct))

    fig, ax = plt.subplots(1, 1, figsize=(14, 8))
    ax.imshow(rgb)

    # Top mask region
    ax.add_patch(Rectangle((0, 0), w, top_px, facecolor="red", alpha=0.35))
    ax.axhline(
        y=top_px,
        color="red",
        linewidth=2,
        linestyle="--",
        label=f"top mask: {top_pct:.0%} ({top_px}px)",
    )

    # Bottom mask region
    ax.add_patch(
        Rectangle((0, bottom_px), w, h - bottom_px, facecolor="red", alpha=0.35)
    )
    ax.axhline(
        y=bottom_px,
        color="orange",
        linewidth=2,
        linestyle="--",
        label=f"bottom mask: {bottom_pct:.0%} ({h - bottom_px}px)",
    )

    ax.set_title(title, fontsize=13)
    ax.legend(loc="upper right", fontsize=10)
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()


for idx, frame in sorted(frames.items()):
    show_hud_overlay(
        frame,
        HUD_TOP_PERCENT,
        HUD_BOTTOM_PERCENT,
        title=f"Frame {idx} — HUD Mask Regions",
    )

## Side-by-Side: Original vs Masked Frame

Compare the raw frame to the masked version that gets fed into the detector.

In [ ]:
sample_frame = frames[SAMPLE_FRAME_INDICES[0]]
masked_frame = apply_hud_mask(sample_frame, HUD_TOP_PERCENT, HUD_BOTTOM_PERCENT)

fig, axes = plt.subplots(1, 2, figsize=(18, 7))

axes[0].imshow(cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original Frame", fontsize=13)
axes[0].set_axis_off()

axes[1].imshow(cv2.cvtColor(masked_frame, cv2.COLOR_BGR2RGB))
axes[1].set_title("After HUD Mask", fontsize=13)
axes[1].set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
out_path = root / "outputs" / "hud_mask_example.png"
out_path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path, dpi=150, bbox_inches="tight")
print(f"Saved → {out_path}")

## Run Detection: With vs Without HUD Mask

Compare detections on the same frame with and without HUD masking enabled to see which false positives the mask eliminates.

In [ ]:
config = load_config(CONFIG_PATH)

detector_masked = PlayerDetector(
    model_name=config["detection"]["model"],
    conf_threshold=config["detection"]["confidence_threshold"],
    iou_threshold=config["detection"]["nms_iou_threshold"],
    device=config["detection"]["device"],
    hud_top=HUD_TOP_PERCENT,
    hud_bottom=HUD_BOTTOM_PERCENT,
    hud_enabled=True,
)

detector_raw = PlayerDetector(
    model_name=config["detection"]["model"],
    conf_threshold=config["detection"]["confidence_threshold"],
    iou_threshold=config["detection"]["nms_iou_threshold"],
    device=config["detection"]["device"],
    hud_enabled=False,
)

In [ ]:
def draw_detections(frame_bgr, detections, title=""):
    """Draw detection bounding boxes on a frame and display it."""
    rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(1, 1, figsize=(14, 8))
    ax.imshow(rgb)

    for det in detections:
        x1, y1, x2, y2, conf = det
        rect = Rectangle(
            (x1, y1),
            x2 - x1,
            y2 - y1,
            linewidth=2,
            edgecolor="lime",
            facecolor="none",
        )
        ax.add_patch(rect)
        ax.text(
            x1,
            y1 - 4,
            f"{conf:.2f}",
            color="lime",
            fontsize=8,
            bbox=dict(boxstyle="round,pad=0.2", facecolor="black", alpha=0.6),
        )

    ax.set_title(f"{title}  ({len(detections)} detections)", fontsize=13)
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()

In [ ]:
for idx, frame in sorted(frames.items()):
    dets_masked = detector_masked.detect(frame)
    dets_raw = detector_raw.detect(frame)

    print(f"--- Frame {idx} ---")
    print(f"  Without mask: {len(dets_raw)} detections")
    print(f"  With mask:    {len(dets_masked)} detections")
    print(f"  Removed:      {len(dets_raw) - len(dets_masked)}")
    print()

    draw_detections(frame, dets_raw, title=f"Frame {idx} — No HUD Mask")
    draw_detections(frame, dets_masked, title=f"Frame {idx} — With HUD Mask")

## Interactive HUD Tuning

Quickly test different `top_percent` / `bottom_percent` values on a single frame to find the right mask boundaries.

In [ ]:
tune_frame = frames[SAMPLE_FRAME_INDICES[0]]

candidates = [
    (0.08, 0.10),
    (0.10, 0.12),
    (0.12, 0.15),
]

fig, axes = plt.subplots(1, len(candidates), figsize=(7 * len(candidates), 7))

for ax, (top, bot) in zip(axes, candidates):
    rgb = cv2.cvtColor(tune_frame, cv2.COLOR_BGR2RGB)
    h, w = rgb.shape[:2]
    top_px = int(h * top)
    bot_px = int(h * (1.0 - bot))

    _ = ax.imshow(rgb)
    _ = ax.add_patch(Rectangle((0, 0), w, top_px, facecolor="red", alpha=0.35))
    _ = ax.axhline(y=top_px, color="red", linewidth=2, linestyle="--")
    _ = ax.add_patch(Rectangle((0, bot_px), w, h - bot_px, facecolor="red", alpha=0.35))
    _ = ax.axhline(y=bot_px, color="orange", linewidth=2, linestyle="--")
    _ = ax.set_title(f"top={top:.0%}  bottom={bot:.0%}", fontsize=12)
    _ = ax.set_axis_off()

_ = plt.suptitle("HUD Mask Candidates", fontsize=14)
plt.tight_layout()
plt.show()